In [1]:
# PTT
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import time
import random
import pymongo
from pymongo import MongoClient
from datetime import datetime
from tomorrow import threads

In [11]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [12]:
def TheTime():
    """抓本機系統時間"""
    ISOTIMEFORMAT = '%Y'
    theTime = datetime.now().strftime(ISOTIMEFORMAT)
    return theTime


In [13]:
def LastPage(board):
    '''抓最後一頁'''
    res = requests.get("https://www.ptt.cc/bbs/"+str(board)+"/index.html",headers=headers,cookies={'over18': '1'})
    soup = BeautifulSoup(res.text)
    pages = soup.find_all("div",class_="btn-group btn-group-paging")[0].find_all("a",class_="btn wide")[1].get("href").strip("/bbs/"+str(board)+"/index .html")
    return int(pages)+1

In [28]:
def GetAllLinks(board,lastpage,stop_page):
    '''抓全部link,輸入的lastpage必須是int'''
    links = []
    while True:
        res = requests.get("https://www.ptt.cc/bbs/"+str(board)+"/index"+str(lastpage)+".html",headers=headers,cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        domain = "https://www.ptt.cc/"
        for i in soup.find_all("div",class_="title"):
            try:
                links.append(domain + i.find("a").get("href"))
            except:
                pass
        lastpage -= 1
        if lastpage == int(stop_page):
            break
    return links
      

In [15]:
# search
def GetSearchLinks(board,search,stop_page=10000):
    '''search是要搜尋的關鍵字 '''
    first_page = 1
    links = []
    while True:
        res = requests.get("https://www.ptt.cc/bbs/"+str(board)+"/search?page="+str(first_page)+"&q="+str(search),headers=headers,cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        try:
            if soup.find("div",class_="bbs-screen bbs-content").text == '404 - Not Found.':
                break
        except:
            pass
        domain = "https://www.ptt.cc/"
        for i in soup.find_all("div",class_="title"):
            try:
                links.append(domain + i.find("a").get("href"))
            except:
                pass
        first_page += 1
        
        if first_page > int(stop_page):
            break
    return links

In [16]:
def GetPTT(response):
    '''板、url、發文日期、作者、title、內容、回文'''
    soup = BeautifulSoup(response.text)
    Board = soup.find("div",class_="article-metaline-right").find("span",class_="article-meta-value").text
    Url = soup.find("link",rel="canonical").get("href")
    CreateTime = soup.find_all("div",class_="article-metaline")[2].find("span",class_="article-meta-value").text
    CreateTime = datetime.strptime(CreateTime,'%a %b %d %H:%M:%S %Y')
    Author = soup.find_all("div",class_="article-metaline")[0].find("span",class_="article-meta-value").text
    Title = soup.find_all("div",class_="article-metaline")[1].find("span",class_="article-meta-value").text

    content_list = []
    for i in soup.find_all("div",class_="bbs-content")[2].contents[4:]:
        if str(i)[:23] == '<span class="f2">※ 發信站:':
            break
        else:
            try:    
                content_list.append(i.text.replace("\n"," "))
            except:
                content_list.append(i.replace("\n"," "))
    Content = " ".join(content_list)

    Reply = []
    for i in range(len(soup.find_all("div",class_="push"))):
        Reaction  = soup.find_all("div",class_="push")[i].contents[0].text.strip(" ")
        Name = soup.find_all("div",class_="push")[i].contents[1].text.strip(" ")
        Comment = soup.find_all("div",class_="push")[i].contents[2].text.strip(": ")
        Reply_dict = {"Name":Name,"Comment":Comment,"Reaction":Reaction}
        Reply.append(Reply_dict)

    ndf = pd.DataFrame(data = [{
                                "Source":"PTT",
                                "Site":Board,
                                "Url":Url,
                                "CreateTime":CreateTime,
                                "Author":Author,
                                "Title":Title,
                                "Content":Content,
                                "Reply":Reply,
                                "ReplyCount":len(Reply)}],
                       columns = ["Source", "Site", "Url", "CreateTime", "Author", "Title", "Content","Reply","ReplyCount"]) 
    
    return ndf


In [8]:
# board = "Gossiping"
# url = 'https://www.ptt.cc/bbs/Gossiping/M.1575128097.A.D41.html'
# res = requests.get(url,headers=headers,cookies={'over18': '1'})
# GetPTT(res)

https://www.ptt.cc/bbs/Gossiping/M.1575128097.A.D41.html


,Source,Site,Url,CreateTime,Author,Title,Content,Reply,ReplyCount
0,PTT,Gossiping,https://www.ptt.cc/bbs/Gossiping/M.1575128097....,2019-11-30 23:34:55,dilson (中肯的話通常不好聽懂嗎),Re: [問卦] 如果被侵略 要打仗 大家願意上戰場嗎,要人民上戰場保護家園 前提是政客說的保護家園 這個家園有讓人民好過嗎? 若薪水低 物...,"[{'Name': 'philip2364', 'Comment': '言論自由不代表你可以...",15


In [17]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        res = requests.get(url, headers=headers,cookies={'over18': '1'})
        return res
    except:
        pass


In [18]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetPTT(response)
            allData.append(ndf)           
        except:
            pass

    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    
    return df


In [19]:
# 測試
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_forum_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

        


In [32]:
# 政治
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.forum_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

In [26]:
links = GetSearchLinks("Gossiping",search="立委",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [27]:
links = GetSearchLinks("Gossiping",search="立法委員",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [ ]:
links = GetSearchLinks("Gossiping",search="大選",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [ ]:
links = GetSearchLinks("Gossiping",search="選舉",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [ ]:
links = GetSearchLinks("Gossiping",search="2020",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [ ]:
links = GetSearchLinks("Gossiping",search="區域",stop_page=20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [35]:
lastpage=LastPage("politics")
links = GetAllLinks(board="politics",lastpage=lastpage,stop_page=lastpage-20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)

In [ ]:
lastpage=LastPage("HatePolitics")
links = GetAllLinks(board="HatePolitics",lastpage=lastpage,stop_page=lastpage-20)
df = CrawlAllNews(links)
dataframe_to_mongo(df)